In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from PyHa.PyHa.statistics import *
from PyHa.PyHa.IsoAutio import *
from PyHa.PyHa.visualizations import *
from PyHa.PyHa.annotation_post_processing import *
from WTS_chunking import annotation_chunker_no_duplicates, extact_split, fill_no_class
import pandas as pd
WTS_data = None

# Weakly to Strongly Labeled Data Generation
If you do not yet have strongly labeled data, run the following script to take weakly labeled data into strongly labeled data

In [4]:
path = "./Data/Test/"
data_output_filename = "wts.csv"
to_filename = True
post_wts_chunk = True
no_bird_from_background = True,
exact = False
manual_df = pd.read_csv("C:/Users/seanh/Desktop/e4e/passive-acoustic-biodiversity/BirdNET-Analyzer/Data/cosmos/cosmos_labeled_data_files_added.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/seanh/Desktop/e4e/passive-acoustic-biodiversity/BirdNET-Analyzer/Data/cosmos/cosmos_labeled_data_files_added.csv'

In [ ]:
# Example parameters for BirdNET
#isolation_parameters = {
#    "model" : "birdnet",
#    "output_path" : "outputs",
#    "lat" : 35.4244,
#    "lon" : -120.7463,
#    "week" : 18,
#    "min_conf" : 0.5,
#    "filetype" : "wav", 
#    "num_predictions" : 1,
#    "write_to_csv" : False
#}

# Example Parameters for Microfaune
#isolation_parameters = {
#     "model" : "microfaune",
#     "technique" : "steinberg",
#     "threshold_type" : "median",
#     "threshold_const" : 2.0,
#     "threshold_min" : 0.0,
#     "window_size" : 2.0,
#     "chunk_size" : 5.0
#}

# Example parameters for TweetyNET
isolation_parameters = {
     "model" : "tweetynet",
     "tweety_output": True,
    "technique" : "steinberg",
     "threshold_type" : "median",
     "threshold_const" : 2.0,
     "threshold_min" : 0.0,
     "window_size" : 2.0,
     "chunk_size" : 5.0
}

##TODO: Test Mircofaune and Birdnet

In [ ]:
automated_df = generate_automated_labels(path,isolation_parameters);

In [ ]:
##TODO: Add birdnet-Lite Labeling system
if (to_filename):
    species_names_df = automated_df["IN FILE"]
    species_names_df = species_names_df.apply(lambda x: " ".join(x.replace("_-_"," - ").split(" - ")[2].strip().split(".")[0].split(" ")[:2]))
    automated_df["MANUAL ID"] = species_names_df

automated_df

In [ ]:
def clean_duration(data):
    data[data["DURATION"] + data["OFFSET"] > data["CLIP LENGTH"]]["DURATION"] = data[data["DURATION"] + data["OFFSET"] > data["CLIP LENGTH"]]["CLIP LENGTH"]
    return data
def fix_overflow(row):
    if (row["DURATION"] + row["OFFSET"] > row["CLIP LENGTH"]):
        row["DURATION"] = row["CLIP LENGTH"] - row["OFFSET"]
    return row
automated_df = automated_df.apply(fix_overflow, axis=1)


In [ ]:
if post_wts_chunk:
    WTS_data = annotation_chunker_no_duplicates(automated_df, 3, no_bird_from_background)
elif exact:
    chunk  = 3
    automated_df = automated_df[["IN FILE", "FOLDER", "OFFSET", "DURATION", "MANUAL ID", "CONFIDENCE", "CLIP LENGTH"]]
    WTS_data = extact_split(automated_df, chunk)
    WTS_data["CHUNK_ID"] = WTS_data["OFFSET"] // chunk
    
    if (no_bird_from_background):
        WTS_data = fill_no_class(WTS_data, chunk)
else:
    WTS_data = automated_df

In [ ]:
WTS_data

In [ ]:
WTS_data.to_csv(data_output_filename)

# Train Models
Select a model to train with WTS data

In [ ]:
import 

In [ ]:
if(WTS_data == None):
    WTS_data = WTS_data.read_csv(data_output_filename)

In [ ]:
classes_of_interest = np.unique(WTS_data["MANUAL ID"])

In [ ]:
manual_df

In [ ]:
WTS_data

In [ ]:
from data_processing import general_training_testing
general_training_testing(WTS_data, manual_df, species_of_interests, model="birdnet",)